In [ ]:
import Pkg; Pkg.activate("..")

In [ ]:
using CSV, DataFrames, DelimitedFiles, Clapeyron, PyCall
import PyPlot; const plt = PyPlot ;

In [ ]:
training_data = CSV.read("saft_vr_mie_validation_data.csv",DataFrame)
num_rows = nrow(training_data)
rename!(training_data,:sat_temperatures_K=>:sat_temp_strings, :sat_pressures_MPa=>:sat_pressure_strings,
        :sat_volumes_liq_m3_per_mol=>:sat_vol_liq_strings,:sat_volumes_vap_m3_per_mol=>:sat_vol_vap_strings)

# Define the column names
array_cols_to_create = [:sat_temperatures_K, :sat_pressures_MPa, :sat_volumes_liq_m3_per_mol, :sat_volumes_vap_m3_per_mol]
string_columns = [:sat_temp_strings, :sat_pressure_strings, :sat_vol_liq_strings, :sat_vol_vap_strings]

# Initialize the columns with zeros
for col in array_cols_to_create
    training_data[!, col] .= fill([0.0], num_rows)
end

# Process and assign the values to the DataFrame
for (col, str_col) in zip(array_cols_to_create, string_columns)
    for i = 1:num_rows
        string_sq_brackets_removed = replace(training_data[i, str_col], r"[][]" => "")
        vector_of_floats = [parse(Float64, val) for val in split(string_sq_brackets_removed, ",")]
        training_data[i, col] = vector_of_floats
    end
end

# Remove the string columns
select!(training_data, Not(string_columns...)) ;

# training_data